In [10]:
import logging
from datetime import datetime
from tqdm.auto import tqdm

from importlib import reload

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: print(f"{datetime.today()} {msg}")

import pandas as pd
import os

import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..'))
sys.path.append(root_dir)


In [2]:
import paramiko

hostname = os.getenv('SSH_HOST')
username = os.getenv('SSH_USERNAME')
password = os.getenv('SSH_PASSWORD')
remote_dir = '/home/graham/audio'

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, password=password, compress=True)

# List files in the remote directory
stdin, stdout, stderr = ssh.exec_command(f'ls {remote_dir}')
remote_files = [f for f in stdout.read().decode().splitlines() if f.endswith(".opus")]

sftp = ssh.open_sftp()



2025-04-05 08:48:13,013 - INFO - Connected (version 2.0, client OpenSSH_9.6)
2025-04-05 08:48:13,201 - INFO - Authentication (password) successful!
2025-04-05 08:48:15,581 - INFO - [chan 1] Opened sftp connection (server version 3)


# Tests

In [13]:
import audio
reload(audio)


audio.train_find_audio_events(sftp, remote_dir)

recording_20250404_011856.opus: 100%|##########| 298k/298k [00:00<00:00, 1.60MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmpmsgqfskt.opus


recording_20250404_013557.opus: 100%|##########| 300k/300k [00:00<00:00, 778kB/s] 


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmpcliy0pgi.opus


recording_20250404_011125.opus: 100%|##########| 299k/299k [00:00<00:00, 1.04MB/s]


...

recording_20250405_010934.opus: 100%|##########| 299k/299k [00:00<00:00, 1.24MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmp_72266_0.opus


recording_20250405_011034.opus: 100%|##########| 299k/299k [00:00<00:00, 1.30MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmp58d1ey_r.opus
Found perfect parameters!
{'window_size': 2048, 'threshold_multiplier': 1.5, 'min_event_duration': 0.2, 'merge_distance': 0.3}

Events found in each file:
recording_20250404_011856.opus: 0 events
recording_20250404_013557.opus: 0 events
recording_20250404_011125.opus: 0 events
recording_20250403_212804.opus: 12 events
  (minimum required: 10)
recording_20250403_212734.opus: 9 events
  (minimum required: 8)
recording_20250405_010934.opus: 0 events
recording_20250405_011034.opus: 1 events
  (minimum required: 1)


{'window_size': 2048,
 'threshold_multiplier': 1.5,
 'min_event_duration': 0.2,
 'merge_distance': 0.3}

In [ ]:
from tqdm.auto import tqdm
import io
import contextlib
from notebooks.Util.DayAndNightOfFinder import day_and_night_of
reload(audio)

bp = audio.best_silence_detection_params

interesting = [f for f in remote_files if day_and_night_of(f.removeprefix('recording_').removesuffix('.opus')) == '2025-04-04']
for idx, i in tqdm(enumerate(interesting), total=len(interesting)):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
        print(i)

        copied = audio.copy_audio_file(sftp, remote_dir, i)
        data_array, ogg, temp_filename = audio.load_audio_as_ogg(copied.getvalue())
        events = audio.find_audio_events(data_array, ogg.frequency, i,
                            window_size=bp['window_size'],
                            threshold_multiplier=bp['threshold_multiplier'],
                            min_event_duration=bp['min_event_duration'],
                            merge_distance=bp['merge_distance'])
        remote_path = audio.save_events_metadata(events, i, sftp, remote_dir)
        print(f"Saved metadata to {remote_path}")


  0%|          | 0/1414 [00:00<?, ?it/s]

2025-04-05 10:39:01,378 - ERROR - Socket exception: An existing connection was forcibly closed by the remote host (10054)
